In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import time

from model import Generator, Discriminator
from config import irgan_config
from data_utils import RecDataset, DataProvider
from evaluation.rec_evaluator import RecEvaluator

In [2]:
epochs = irgan_config.epochs
batch_size = irgan_config.batch_size
epochs_d = irgan_config.epochs_d
epochs_g = irgan_config.epochs_g
emb_dim = irgan_config.emb_dim
eta_G = irgan_config.eta_G
eta_D = irgan_config.eta_D
device = irgan_config.device
weight_decay_g = irgan_config.weight_decay_g
weight_decay_d = irgan_config.weight_decay_d

In [3]:
# Hyper-parameters and datset-specific parameters
rec_dataset = RecDataset(irgan_config.dir_path)
all_users = rec_dataset.get_users()
all_items = rec_dataset.get_items()
num_users = rec_dataset.get_num_users()
num_items = rec_dataset.get_num_items()
bought_mask = rec_dataset.get_bought_mask().to(device)
eval_dict = rec_dataset.get_interaction_records("test")
train_ui = rec_dataset.get_user_item_pairs()

In [4]:
dp = DataProvider(device)
evaluator = RecEvaluator(eval_dict, None, device)

In [5]:
writer_dis = SummaryWriter("runs/IRGAN-SGD-dns-gen-sns/Discriminator")
writer_gen = SummaryWriter("runs/IRGAN-SGD-dns-gen-sns/Generator")

In [6]:
G = Generator(num_users, num_items, emb_dim, bought_mask)
D = Discriminator(num_users, num_items, emb_dim)
G = G.to(device)
D = D.to(device)

loss_D = nn.BCELoss()
optimizer_G = torch.optim.SGD(G.parameters(), lr = eta_G)
optimizer_D = torch.optim.SGD(D.parameters(), lr = eta_D, weight_decay = weight_decay_d)

In [7]:
pretrained_model = torch.load("./pretrained_models/pretrained_model_sns.pkl",map_location=irgan_config.device)
G.load_state_dict(pretrained_model)

<All keys matched successfully>

In [ ]:
for epoch in range(epochs):
    print(
            "[Epoch %d/%d]"
            % (epoch+1, epochs)
        )  
    time_epoch_start = time.time()
    
    # ---------------------
    #  Train Discriminator
    # ---------------------   
    D.train()
    G.eval()
    for epoch_d in range(epochs_d):
        time_d_start = time.time()
        if(epoch_d % 5 == 0):
            train_set_d = dp.prepare_data_for_discriminator(G, train_ui, batch_size = batch_size)
        loss_d_epoch = 0
        for users, items, labels in train_set_d:
            dis_score = D(users,items)
            loss_d = loss_D(dis_score, labels)
            loss_d_epoch += loss_d.item()
            optimizer_D.zero_grad()
            loss_d.backward()
            optimizer_D.step()
            
        time_d_end = time.time()
        loss_d_epoch /= len(train_set_d)
        print(
            "\t[Discriminator][Epochs %d/%d] [D epoch loss: %6.5f] [Time:%6.5f] "
            % (epoch_d+1, epochs_d, loss_d_epoch, time_d_end - time_d_start)
        )
        writer_dis.add_scalar('Loss/Discriminator', loss_d_epoch, epoch*epochs_d + epoch_d)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(D, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_dis.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MAP', map10, epoch*epochs_d + epoch_d)
            writer_dis.add_scalar('Metrics/MRR', mrr10, epoch*epochs_d + epoch_d)
        
    # --------------------- 
    #  Train Generator
    #
    # For generator
    #   Generate K user-item pairs
    #   Leveraging Policy Gradient to update parameters of generator
    #
    # --------------------- 
    D.eval()
    G.train()
    for epoch_g in range(epochs_g):
        time_g_start = time.time()
        train_set_g = dp.prepare_data_for_generator(all_users, batch_size)
        loss_g_epoch = 0
        for fake_users, in train_set_g:
            fake_items, fake_probs, fake_p_n = G.sample_items_for_users(fake_users, k = 256,\
                                                                        temperature=1, lambda_bought=0.2)
            fake_users = fake_users.view(-1,1).expand_as(fake_items).contiguous()
            fake_users = fake_users.view(-1)
            fake_items = fake_items.view(-1)
            fake_probs = fake_probs.view(-1)
            fake_p_n = fake_p_n.view(-1)
            
            log_fake_probs = torch.log(fake_probs.clamp(1e-8))
            fake_probs = fake_probs.detach()
            fake_p_n = fake_p_n.detach()
            
            reward = (2 * D(fake_users, fake_items) - 1).detach()*(fake_probs/fake_p_n)
            loss_g = -torch.mean(log_fake_probs*reward)
            loss_g_epoch += loss_g
            
            optimizer_G.zero_grad()  
            loss_g.backward() 
            optimizer_G.step()
        time_g_end = time.time()
        print(
            "\t[Generator][Epochs %d/%d] [G epoch loss: %6.5f] [Time:%6.5f]"
            % (epoch_g + 1, epochs_g, loss_g_epoch, time_g_end - time_g_start )
        ) 
        writer_gen.add_scalar('Loss/Generator', loss_g_epoch, epoch*epochs_g + epoch_g)
        with torch.no_grad():
            res = evaluator.top_k_evaluation(G, [3,5,10])    
            ndcg3, precision3, hit3, map3, mrr3 = res[0]
            ndcg5, precision5, hit5, map5, mrr5 = res[1]
            ndcg10, precision10, hit10, map10, mrr10 = res[2]
            writer_gen.add_scalar('Metrics/NDCG@3', ndcg3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@5', ndcg5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/NDCG@10', ndcg10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@3', precision3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@5', precision5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Precision@10', precision10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@3', hit3, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@5', hit5, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/Hit@10', hit10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MAP', map10, epoch*epochs_g + epoch_g)
            writer_gen.add_scalar('Metrics/MRR', mrr10, epoch*epochs_g + epoch_g)
    time_epoch_end = time.time()
writer_dis.close()    
writer_gen.close()

[Epoch 1/200]
	[Discriminator][Epochs 1/100] [D epoch loss: 0.68408] [Time:5.56429] 
	[Discriminator][Epochs 2/100] [D epoch loss: 0.66722] [Time:4.69720] 
	[Discriminator][Epochs 3/100] [D epoch loss: 0.65206] [Time:4.66549] 
	[Discriminator][Epochs 4/100] [D epoch loss: 0.63837] [Time:4.66023] 
	[Discriminator][Epochs 5/100] [D epoch loss: 0.62594] [Time:4.62982] 
	[Discriminator][Epochs 6/100] [D epoch loss: 0.61456] [Time:3.39265] 
	[Discriminator][Epochs 7/100] [D epoch loss: 0.60417] [Time:3.35577] 
	[Discriminator][Epochs 8/100] [D epoch loss: 0.59461] [Time:3.35962] 
	[Discriminator][Epochs 9/100] [D epoch loss: 0.58578] [Time:3.36639] 
	[Discriminator][Epochs 10/100] [D epoch loss: 0.57759] [Time:3.39514] 
	[Discriminator][Epochs 11/100] [D epoch loss: 0.57000] [Time:3.40251] 
	[Discriminator][Epochs 12/100] [D epoch loss: 0.56289] [Time:3.37078] 
	[Discriminator][Epochs 13/100] [D epoch loss: 0.55625] [Time:3.35940] 
	[Discriminator][Epochs 14/100] [D epoch loss: 0.55001] [Ti

	[Generator][Epochs 16/50] [G epoch loss: -8.92058] [Time:0.03462]
	[Generator][Epochs 17/50] [G epoch loss: -8.90446] [Time:0.03527]
	[Generator][Epochs 18/50] [G epoch loss: -8.83655] [Time:0.03391]
	[Generator][Epochs 19/50] [G epoch loss: -8.92074] [Time:0.03796]
	[Generator][Epochs 20/50] [G epoch loss: -8.95460] [Time:0.03328]
	[Generator][Epochs 21/50] [G epoch loss: -8.93487] [Time:0.03626]
	[Generator][Epochs 22/50] [G epoch loss: -8.88488] [Time:0.03536]
	[Generator][Epochs 23/50] [G epoch loss: -8.85928] [Time:0.04021]
	[Generator][Epochs 24/50] [G epoch loss: -8.86326] [Time:0.03540]
	[Generator][Epochs 25/50] [G epoch loss: -8.79799] [Time:0.03470]
	[Generator][Epochs 26/50] [G epoch loss: -8.83851] [Time:0.03499]
	[Generator][Epochs 27/50] [G epoch loss: -8.78726] [Time:0.03419]
	[Generator][Epochs 28/50] [G epoch loss: -8.86283] [Time:0.03513]
	[Generator][Epochs 29/50] [G epoch loss: -8.92189] [Time:0.03449]
	[Generator][Epochs 30/50] [G epoch loss: -8.91748] [Time:0.03

	[Discriminator][Epochs 83/100] [D epoch loss: 0.37805] [Time:4.69627] 
	[Discriminator][Epochs 84/100] [D epoch loss: 0.37786] [Time:4.66758] 
	[Discriminator][Epochs 85/100] [D epoch loss: 0.37767] [Time:4.65509] 
	[Discriminator][Epochs 86/100] [D epoch loss: 0.37750] [Time:3.40217] 
	[Discriminator][Epochs 87/100] [D epoch loss: 0.37732] [Time:3.37265] 
	[Discriminator][Epochs 88/100] [D epoch loss: 0.37713] [Time:3.34536] 
	[Discriminator][Epochs 89/100] [D epoch loss: 0.37695] [Time:3.38703] 
	[Discriminator][Epochs 90/100] [D epoch loss: 0.37677] [Time:3.36055] 
	[Discriminator][Epochs 91/100] [D epoch loss: 0.37654] [Time:4.71420] 
	[Discriminator][Epochs 92/100] [D epoch loss: 0.37636] [Time:4.72065] 
	[Discriminator][Epochs 93/100] [D epoch loss: 0.37619] [Time:4.63983] 
	[Discriminator][Epochs 94/100] [D epoch loss: 0.37601] [Time:4.69565] 
	[Discriminator][Epochs 95/100] [D epoch loss: 0.37584] [Time:4.68633] 
	[Discriminator][Epochs 96/100] [D epoch loss: 0.37571] [Time:3.

	[Discriminator][Epochs 50/100] [D epoch loss: 0.37212] [Time:4.72720] 
	[Discriminator][Epochs 51/100] [D epoch loss: 0.37206] [Time:4.69450] 
	[Discriminator][Epochs 52/100] [D epoch loss: 0.37194] [Time:4.68376] 
	[Discriminator][Epochs 53/100] [D epoch loss: 0.37183] [Time:4.69184] 
	[Discriminator][Epochs 54/100] [D epoch loss: 0.37171] [Time:4.65068] 
	[Discriminator][Epochs 55/100] [D epoch loss: 0.37160] [Time:4.69906] 
	[Discriminator][Epochs 56/100] [D epoch loss: 0.37156] [Time:7.13535] 
	[Discriminator][Epochs 57/100] [D epoch loss: 0.37145] [Time:4.65539] 
	[Discriminator][Epochs 58/100] [D epoch loss: 0.37133] [Time:4.37050] 
	[Discriminator][Epochs 59/100] [D epoch loss: 0.37122] [Time:4.40145] 
	[Discriminator][Epochs 60/100] [D epoch loss: 0.37111] [Time:4.50612] 
	[Discriminator][Epochs 61/100] [D epoch loss: 0.37098] [Time:7.07857] 
	[Discriminator][Epochs 62/100] [D epoch loss: 0.37088] [Time:4.69536] 
	[Discriminator][Epochs 63/100] [D epoch loss: 0.37077] [Time:4.

	[Discriminator][Epochs 17/100] [D epoch loss: 0.36849] [Time:4.43755] 
	[Discriminator][Epochs 18/100] [D epoch loss: 0.36841] [Time:4.59027] 
	[Discriminator][Epochs 19/100] [D epoch loss: 0.36833] [Time:4.62225] 
	[Discriminator][Epochs 20/100] [D epoch loss: 0.36825] [Time:4.41892] 
	[Discriminator][Epochs 21/100] [D epoch loss: 0.36807] [Time:4.62751] 
	[Discriminator][Epochs 22/100] [D epoch loss: 0.36799] [Time:4.64865] 
	[Discriminator][Epochs 23/100] [D epoch loss: 0.36791] [Time:4.66938] 
	[Discriminator][Epochs 24/100] [D epoch loss: 0.36783] [Time:4.68208] 
	[Discriminator][Epochs 25/100] [D epoch loss: 0.36776] [Time:4.67738] 
	[Discriminator][Epochs 26/100] [D epoch loss: 0.36768] [Time:4.66951] 
	[Discriminator][Epochs 27/100] [D epoch loss: 0.36760] [Time:4.68336] 
	[Discriminator][Epochs 28/100] [D epoch loss: 0.36753] [Time:4.68350] 
	[Discriminator][Epochs 29/100] [D epoch loss: 0.36745] [Time:4.74246] 
	[Discriminator][Epochs 30/100] [D epoch loss: 0.36737] [Time:4.

...

	[Generator][Epochs 44/50] [G epoch loss: -10.33287] [Time:0.03566]
	[Generator][Epochs 45/50] [G epoch loss: -10.39726] [Time:0.03640]
	[Generator][Epochs 46/50] [G epoch loss: -10.36110] [Time:0.03725]
	[Generator][Epochs 47/50] [G epoch loss: -10.53958] [Time:0.03707]
	[Generator][Epochs 48/50] [G epoch loss: -10.54410] [Time:0.03562]
	[Generator][Epochs 49/50] [G epoch loss: -10.50477] [Time:0.03638]
	[Generator][Epochs 50/50] [G epoch loss: -10.52030] [Time:0.03721]
[Epoch 22/200]
	[Discriminator][Epochs 1/100] [D epoch loss: 0.37821] [Time:6.06277] 
	[Discriminator][Epochs 2/100] [D epoch loss: 0.37819] [Time:4.66465] 
	[Discriminator][Epochs 3/100] [D epoch loss: 0.37818] [Time:4.67538] 
	[Discriminator][Epochs 4/100] [D epoch loss: 0.37817] [Time:4.65098] 
	[Discriminator][Epochs 5/100] [D epoch loss: 0.37815] [Time:4.72028] 
	[Discriminator][Epochs 6/100] [D epoch loss: 0.37809] [Time:7.03932] 
	[Discriminator][Epochs 7/100] [D epoch loss: 0.37808] [Time:4.69072] 
	[Discrimina

	[Generator][Epochs 9/50] [G epoch loss: -10.47905] [Time:0.03982]
	[Generator][Epochs 10/50] [G epoch loss: -10.64125] [Time:0.03499]
	[Generator][Epochs 11/50] [G epoch loss: -10.54390] [Time:0.03515]
	[Generator][Epochs 12/50] [G epoch loss: -10.59939] [Time:0.03579]
	[Generator][Epochs 13/50] [G epoch loss: -10.55098] [Time:0.03493]
	[Generator][Epochs 14/50] [G epoch loss: -10.56358] [Time:0.03571]
	[Generator][Epochs 15/50] [G epoch loss: -10.63747] [Time:0.03678]
	[Generator][Epochs 16/50] [G epoch loss: -10.63303] [Time:0.03604]
	[Generator][Epochs 17/50] [G epoch loss: -10.49446] [Time:0.04164]
	[Generator][Epochs 18/50] [G epoch loss: -10.57729] [Time:0.03582]
	[Generator][Epochs 19/50] [G epoch loss: -10.36642] [Time:0.03586]
	[Generator][Epochs 20/50] [G epoch loss: -10.65114] [Time:0.03598]
	[Generator][Epochs 21/50] [G epoch loss: -10.52080] [Time:0.03678]
	[Generator][Epochs 22/50] [G epoch loss: -10.54443] [Time:0.03666]
	[Generator][Epochs 23/50] [G epoch loss: -10.583

	[Discriminator][Epochs 76/100] [D epoch loss: 0.37934] [Time:6.77029] 
	[Discriminator][Epochs 77/100] [D epoch loss: 0.37933] [Time:4.70068] 
	[Discriminator][Epochs 78/100] [D epoch loss: 0.37932] [Time:4.65851] 
	[Discriminator][Epochs 79/100] [D epoch loss: 0.37931] [Time:4.68639] 
	[Discriminator][Epochs 80/100] [D epoch loss: 0.37929] [Time:4.67776] 
	[Discriminator][Epochs 81/100] [D epoch loss: 0.37917] [Time:3.48508] 
	[Discriminator][Epochs 82/100] [D epoch loss: 0.37916] [Time:3.40861] 
	[Discriminator][Epochs 83/100] [D epoch loss: 0.37915] [Time:3.40811] 
	[Discriminator][Epochs 84/100] [D epoch loss: 0.37914] [Time:3.40850] 
	[Discriminator][Epochs 85/100] [D epoch loss: 0.37913] [Time:3.37790] 
	[Discriminator][Epochs 86/100] [D epoch loss: 0.37906] [Time:4.67293] 
	[Discriminator][Epochs 87/100] [D epoch loss: 0.37905] [Time:4.72384] 
	[Discriminator][Epochs 88/100] [D epoch loss: 0.37904] [Time:4.71586] 
	[Discriminator][Epochs 89/100] [D epoch loss: 0.37903] [Time:4.

	[Discriminator][Epochs 43/100] [D epoch loss: 0.37995] [Time:4.71854] 
	[Discriminator][Epochs 44/100] [D epoch loss: 0.37994] [Time:4.66900] 
	[Discriminator][Epochs 45/100] [D epoch loss: 0.37993] [Time:4.66414] 
	[Discriminator][Epochs 46/100] [D epoch loss: 0.37988] [Time:4.71132] 
	[Discriminator][Epochs 47/100] [D epoch loss: 0.37987] [Time:4.69393] 
	[Discriminator][Epochs 48/100] [D epoch loss: 0.37986] [Time:4.69736] 
	[Discriminator][Epochs 49/100] [D epoch loss: 0.37984] [Time:4.69256] 
	[Discriminator][Epochs 50/100] [D epoch loss: 0.37983] [Time:4.70975] 
	[Discriminator][Epochs 51/100] [D epoch loss: 0.37983] [Time:4.68113] 
	[Discriminator][Epochs 52/100] [D epoch loss: 0.37982] [Time:4.67591] 
	[Discriminator][Epochs 53/100] [D epoch loss: 0.37981] [Time:4.71753] 
	[Discriminator][Epochs 54/100] [D epoch loss: 0.37979] [Time:4.69966] 
	[Discriminator][Epochs 55/100] [D epoch loss: 0.37978] [Time:4.67950] 
	[Discriminator][Epochs 56/100] [D epoch loss: 0.37975] [Time:4.

	[Discriminator][Epochs 10/100] [D epoch loss: 0.38155] [Time:4.70464] 
	[Discriminator][Epochs 11/100] [D epoch loss: 0.38155] [Time:7.10517] 
	[Discriminator][Epochs 12/100] [D epoch loss: 0.38154] [Time:4.70606] 
	[Discriminator][Epochs 13/100] [D epoch loss: 0.38153] [Time:4.67448] 
	[Discriminator][Epochs 14/100] [D epoch loss: 0.38152] [Time:4.67869] 
	[Discriminator][Epochs 15/100] [D epoch loss: 0.38150] [Time:4.70651] 
	[Discriminator][Epochs 16/100] [D epoch loss: 0.38150] [Time:4.71047] 
	[Discriminator][Epochs 17/100] [D epoch loss: 0.38148] [Time:4.69944] 
	[Discriminator][Epochs 18/100] [D epoch loss: 0.38147] [Time:4.67018] 
	[Discriminator][Epochs 19/100] [D epoch loss: 0.38146] [Time:4.69066] 
	[Discriminator][Epochs 20/100] [D epoch loss: 0.38145] [Time:4.67008] 
	[Discriminator][Epochs 21/100] [D epoch loss: 0.38151] [Time:3.42805] 
	[Discriminator][Epochs 22/100] [D epoch loss: 0.38150] [Time:3.38773] 
	[Discriminator][Epochs 23/100] [D epoch loss: 0.38149] [Time:3.